In [1]:
keypoints = [[
            [
                19.14722442626953,
                41.54705810546875,
                0.6361105442047119
            ],
            [
                16.519424438476562,
                41.1151123046875,
                0.5233421325683594
            ],
            [
                16.265243530273438,
                40.67802429199219,
                0.727553129196167
            ],
            [
                14.557487487792969,
                26.705810546875,
                0.7311871647834778
            ],
            [
                15.869911193847656,
                39.14404296875,
                0.736687958240509
            ],
            [
                32.035789489746094,
                11.601188659667969,
                0.7329443693161011
            ],
            [
                36.59025573730469,
                45.17820739746094,
                0.6948325634002686
            ],
            [
                56.107818603515625,
                -0.47245025634765625,
                0.8378121852874756
            ],
            [
                64.528076171875,
                50.140960693359375,
                0.7974252104759216
            ],
            [
                73.05470275878906,
                6.385490417480469,
                0.7348101139068604
            ],
            [
                80.088623046875,
                56.107505798339844,
                0.7567539215087891
            ],
            [
                82.24462890625,
                17.98583984375,
                0.5630366206169128
            ],
            [
                79.56486511230469,
                38.24049377441406,
                0.6257514357566833
            ],
            [
                112.8123779296875,
                26.198753356933594,
                0.5593718886375427
            ],
            [
                109.5264892578125,
                45.56861877441406,
                0.7608657479286194
            ],
            [
                134.3271484375,
                26.677894592285156,
                0.4632413387298584
            ],
            [
                145.1158447265625,
                53.968902587890625,
                0.702558159828186
            ],
            [
                148.65127563476562,
                29.858623504638672,
                0.5457552671432495
            ],
            [
                148.32174682617188,
                26.223678588867188,
                0.5586147904396057
            ],
            [
                137.65902709960938,
                26.38121795654297,
                0.3840145468711853
            ],
            [
                151.332763671875,
                53.56022644042969,
                0.4267081320285797
            ],
            [
                151.53573608398438,
                57.077781677246094,
                0.47282281517982483
            ],
            [
                150.9775848388672,
                54.12395477294922,
                0.5943477153778076
            ],
            [
                16.35034942626953,
                38.06065368652344,
                0.5013765692710876
            ],
            [
                17.580490112304688,
                38.454345703125,
                0.5064101219177246
            ],
            [
                18.738082885742188,
                38.34899139404297,
                0.4565489590167999
            ],
            [
                19.407150268554688,
                38.174102783203125,
                0.41497644782066345
            ],
            [
                19.71759796142578,
                37.68821716308594,
                0.40979036688804626
            ],
            [
                20.06946563720703,
                37.42340087890625,
                0.40824487805366516
            ],
            [
                20.867637634277344,
                37.401206970214844,
                0.38736870884895325
            ],
            [
                21.930580139160156,
                37.71540832519531,
                0.34737250208854675
            ],
            [
                22.616668701171875,
                38.1954345703125,
                0.3357579708099365
            ],
            [
                22.55828857421875,
                38.59521484375,
                0.3274238705635071
            ],
            [
                22.123779296875,
                39.167633056640625,
                0.32240769267082214
            ],
            [
                21.43603515625,
                39.484901428222656,
                0.35496386885643005
            ],
            [
                20.430152893066406,
                39.546485900878906,
                0.33642613887786865
            ],
            [
                19.612281799316406,
                39.73468017578125,
                0.35093367099761963
            ],
            [
                18.90740966796875,
                39.81056213378906,
                0.3793486952781677
            ],
            [
                18.19805908203125,
                39.877838134765625,
                0.41434964537620544
            ],
            [
                16.991744995117188,
                39.83770751953125,
                0.43770360946655273
            ],
            [
                15.790214538574219,
                39.335166931152344,
                0.587323784828186
            ],
            [
                15.424285888671875,
                39.32572937011719,
                0.6084451079368591
            ],
            [
                15.297531127929688,
                39.52162170410156,
                0.5995973944664001
            ],
            [
                15.474617004394531,
                39.5797119140625,
                0.6064264178276062
            ],
            [
                15.680130004882812,
                39.712135314941406,
                0.5820018649101257
            ],
            [
                15.779464721679688,
                39.7969970703125,
                0.5466236472129822
            ],
            [
                15.754753112792969,
                39.766510009765625,
                0.5003768801689148
            ],
            [
                15.623260498046875,
                39.666481018066406,
                0.47396352887153625
            ],
            [
                15.775833129882812,
                39.50347900390625,
                0.47391393780708313
            ],
            [
                16.084251403808594,
                39.61952209472656,
                0.4479454457759857
            ],
            [
                16.752410888671875,
                40.176483154296875,
                0.5266917943954468
            ],
            [
                17.777984619140625,
                40.6724853515625,
                0.5378815531730652
            ],
            [
                18.548011779785156,
                40.9940185546875,
                0.5659936666488647
            ],
            [
                19.207992553710938,
                41.03582000732422,
                0.5166782140731812
            ],
            [
                19.545326232910156,
                40.13499450683594,
                0.496908038854599
            ],
            [
                19.69306182861328,
                40.281829833984375,
                0.4886445701122284
            ],
            [
                19.788116455078125,
                40.41283416748047,
                0.4818514585494995
            ],
            [
                19.849098205566406,
                40.579978942871094,
                0.491659551858902
            ],
            [
                19.85950469970703,
                40.66920471191406,
                0.4764295220375061
            ],
            [
                16.759803771972656,
                39.716148376464844,
                0.5855250358581543
            ],
            [
                16.511306762695312,
                39.77252197265625,
                0.6070062518119812
            ],
            [
                16.447540283203125,
                39.81394958496094,
                0.7227150201797485
            ],
            [
                16.62915802001953,
                39.87953186035156,
                0.5966925024986267
            ],
            [
                16.682350158691406,
                39.972740173339844,
                0.6075878143310547
            ],
            [
                16.84154510498047,
                39.889892578125,
                0.5946696996688843
            ],
            [
                16.730491638183594,
                40.094512939453125,
                0.48927736282348633
            ],
            [
                16.759719848632812,
                40.51789093017578,
                0.5310587882995605
            ],
            [
                16.65973663330078,
                40.150390625,
                0.4634384214878082
            ],
            [
                16.801589965820312,
                40.16119384765625,
                0.42900991439819336
            ],
            [
                16.78899383544922,
                40.3148193359375,
                0.4827474355697632
            ],
            [
                16.913223266601562,
                40.23698425292969,
                0.5205807685852051
            ],
            [
                20.56822967529297,
                39.61157989501953,
                0.44239282608032227
            ],
            [
                20.544174194335938,
                39.827178955078125,
                0.44982483983039856
            ],
            [
                20.520980834960938,
                39.95787048339844,
                0.44890496134757996
            ],
            [
                20.611419677734375,
                39.95603942871094,
                0.4432780146598816
            ],
            [
                20.613845825195312,
                39.99409484863281,
                0.43912577629089355
            ],
            [
                20.780807495117188,
                39.94776153564453,
                0.42441684007644653
            ],
            [
                20.878219604492188,
                39.805625915527344,
                0.4115045666694641
            ],
            [
                21.16889190673828,
                39.62882995605469,
                0.413682222366333
            ],
            [
                21.228347778320312,
                39.53026580810547,
                0.41569942235946655
            ],
            [
                21.195907592773438,
                39.32939147949219,
                0.4205598831176758
            ],
            [
                21.023269653320312,
                39.27922058105469,
                0.42873072624206543
            ],
            [
                20.726287841796875,
                39.33869934082031,
                0.4373016059398651
            ],
            [
                20.590194702148438,
                39.585357666015625,
                0.44399234652519226
            ],
            [
                20.716957092285156,
                39.66273498535156,
                0.43877243995666504
            ],
            [
                20.868637084960938,
                39.82124328613281,
                0.4402969181537628
            ],
            [
                20.94976043701172,
                39.76885986328125,
                0.42149046063423157
            ],
            [
                20.87396240234375,
                39.761749267578125,
                0.4154509902000427
            ],
            [
                21.04938507080078,
                39.64678955078125,
                0.41725510358810425
            ],
            [
                21.013839721679688,
                39.61615753173828,
                0.42649346590042114
            ],
            [
                20.780120849609375,
                39.45484924316406,
                0.43254825472831726
            ],
            [
                74.27032470703125,
                6.843284606933594,
                0.6463411450386047
            ],
            [
                76.05111694335938,
                7.713603973388672,
                0.5596440434455872
            ],
            [
                77.66990661621094,
                8.73883056640625,
                0.513430655002594
            ],
            [
                79.86285400390625,
                9.191116333007812,
                0.5026413798332214
            ],
            [
                81.95782470703125,
                9.237091064453125,
                0.4836479127407074
            ],
            [
                80.27291870117188,
                8.858291625976562,
                0.5682715773582458
            ],
            [
                81.76730346679688,
                9.131214141845703,
                0.5915399193763733
            ],
            [
                83.04220581054688,
                9.281139373779297,
                0.5806824564933777
            ],
            [
                83.89340209960938,
                9.568008422851562,
                0.5646781921386719
            ],
            [
                81.095947265625,
                8.60696792602539,
                0.5876467227935791
            ],
            [
                82.81040954589844,
                8.879753112792969,
                0.5809720754623413
            ],
            [
                83.91366577148438,
                9.104270935058594,
                0.577666163444519
            ],
            [
                84.70303344726562,
                9.4508056640625,
                0.5840726494789124
            ],
            [
                81.56814575195312,
                8.234134674072266,
                0.5911293029785156
            ],
            [
                83.17120361328125,
                8.663742065429688,
                0.5759355425834656
            ],
            [
                84.29095458984375,
                8.968799591064453,
                0.5651621222496033
            ],
            [
                85.23849487304688,
                9.214714050292969,
                0.5779207348823547
            ],
            [
                81.9169921875,
                7.862827301025391,
                0.5913700461387634
            ],
            [
                83.30784606933594,
                8.2611083984375,
                0.5691273808479309
            ],
            [
                84.37641906738281,
                8.565872192382812,
                0.5764952898025513
            ],
            [
                85.2044677734375,
                8.763763427734375,
                0.5815722346305847
            ],
            [
                81.04278564453125,
                55.99292755126953,
                0.7833564281463623
            ],
            [
                80.78387451171875,
                56.19825744628906,
                0.7761596441268921
            ],
            [
                80.71568298339844,
                56.34027099609375,
                0.6677371859550476
            ],
            [
                81.23677062988281,
                56.47178649902344,
                0.6520186066627502
            ],
            [
                82.32620239257812,
                56.75645446777344,
                0.6395465135574341
            ],
            [
                81.2567138671875,
                57.21281433105469,
                0.6988813877105713
            ],
            [
                82.07014465332031,
                57.48805236816406,
                0.6498682498931885
            ],
            [
                82.93269348144531,
                57.574951171875,
                0.6627339720726013
            ],
            [
                83.44767761230469,
                57.57331848144531,
                0.6522102952003479
            ],
            [
                82.40966796875,
                57.51969909667969,
                0.7197129726409912
            ],
            [
                83.34367370605469,
                57.83061981201172,
                0.7098948359489441
            ],
            [
                83.92631530761719,
                57.79972839355469,
                0.7213175892829895
            ],
            [
                84.30413818359375,
                57.674102783203125,
                0.6861398816108704
            ],
            [
                83.31878662109375,
                57.63861083984375,
                0.7274050116539001
            ],
            [
                84.06600952148438,
                57.91350555419922,
                0.7325955629348755
            ],
            [
                84.61749267578125,
                57.91253662109375,
                0.7349721193313599
            ],
            [
                85.0333251953125,
                57.773040771484375,
                0.7126349210739136
            ],
            [
                84.05322265625,
                57.685699462890625,
                0.7346133589744568
            ],
            [
                84.72367858886719,
                57.878517150878906,
                0.7327976226806641
            ],
            [
                85.25006103515625,
                57.88104248046875,
                0.7288830876350403
            ],
            [
                85.59730529785156,
                57.787261962890625,
                0.7259737253189087
            ]
        ]
    ]

In [4]:
len(keypoints[0])

133